In [1]:
import os
import sys
import asyncio
import time

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [2]:
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/sashabrownsberger/ospl.xml"

In [3]:
os.path.exists('/home/sashabrownsberger/ospl.xml')

True

In [4]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [5]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

Read historical data in 0.00 sec
Read historical data in 0.18 sec
Read historical data in 0.66 sec
Read historical data in 1.11 sec
Read historical data in 1.36 sec
Read historical data in 1.88 sec
Read historical data in 2.01 sec
Read historical data in 2.36 sec
Read historical data in 3.29 sec
Read historical data in 4.65 sec
Read historical data in 8.02 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 31 messages


[[None, None, None, None, None, None, None], [None, None, None, None]]

RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 35 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 66 messages


In [6]:
#Make sure to inform your team members that you are going to enable the attcs and latiss. 
await attcs.enable()
await latiss.enable()

Gathering settings.
No settings for atcamera.
Using  from settingVersions event.
No settings for atspectrograph.
Using current from settingVersions event.
No settings for atheaderservice.
Couldn't get settingVersions event. Using empty settings.
No settings for atarchiver.
Using normal from settingVersions event.
Settings versions: {'atcamera': '', 'atspectrograph': 'current', 'atheaderservice': '', 'atarchiver': 'normal'}
Enabling all components
Enabling  atcamera
Enabling  atspectrograph
Enabling  atheaderservice
Enabling  atarchiver
[atcamera]::[<State.ENABLED: 2>]
[atspectrograph]::[<State.ENABLED: 2>]
[atheaderservice]::[<State.ENABLED: 2>]
[atarchiver]::[<State.ENABLED: 2>]
All components enabled.


In [7]:
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
BIAS 0004 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200001 2020031200002 2020031200003 2020031200004 2020031200005
 2020031200006 2020031200007 2020031200008 2020031200009 2020031200010

In [8]:
import os
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend("firefly")
os.environ['FIREFLY_URL'] = "http://firefly01.cp.lsst.org:8080/firefly/"

RemoteEvent(ATMCS, 0, target) falling behind; read 26 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 26 messages


In [9]:
from lsst.ip.isr.isrTask import IsrTask

isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = False
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doAddDistortionModel = False
isrConfig.doWrite = False

isrTask = IsrTask(config=isrConfig)

/opt/lsst/software/stack/python/miniconda3-4.7.12/envs/lsst-scipipe-984c9f7/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Config field doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.


RemoteEvent(ATMCS, 0, target) falling behind; read 22 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 22 messages


In [12]:
butler = dafPersist.Butler("/project/shared/auxTel")
dataId = dict(dayObs="2020-03-12") #Your observing date 

RemoteEvent(ATMCS, 0, target) falling behind; read 22 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 23 messages


In [ ]:
dataId["seqNum"] = 235 #Your sequence number (the last digits after the dates and. trailing 0's of the data ids printed above)

try:
    bias
except NameError:
    bias = butler.get('bias', dataId)

exp = butler.get('raw', dataId)
exp = isrTask.run(exp, bias=bias).exposure

In [14]:
dataId["seqNum"] = 1
exp = butler.get('raw', dataId)

Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000001.yaml
AT_O_20200312_000001: Forcing detector serial to ITL-3800C-068
AT_O_20200312_000001: Forcing blank RADESYS to 'ICRS'
Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000001.yaml
AT_O_20200312_000001: Forcing detector serial to ITL-3800C-068
Using translation class LSST_LATISS
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 38 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 39 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) fal

In [15]:
disp = afwDisplay.Display(1, name='sample', reopenPlot=True)
disp.scale('asinh', 'zscale')
disp.mtv(exp, title=f"{dataId}")

Starting new HTTPS connection (1): summit-lsp.lsst.codes:443
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.co